In [6]:
import pennylane as qml
from pennylane import numpy as np
 
# Define the molecule
symbols  = ['H', 'H', 'H', 'H']
r_bohr = 1.8897259886
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])  # Coordinates of your molecule
 
electrons = 4
charge = 0
active_orbitals = 4 #Spatial orbitals
active_electrons = 4
 
# Build the electronic Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis = "sto-6g", method="pyscf",active_electrons=electrons, active_orbitals=active_orbitals)
 
# Define the HF state
hf_state = qml.qchem.hf_state(active_electrons, qubits)
 
# Generate single and double excitations
singles, doubles = qml.qchem.excitations(active_electrons, qubits)
#print('singles are', singles)
print('doubles are', doubles)
 
# Map excitations to the wires the UCCSD circuit will act on
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)
print('d wires are', d_wires)
 
# Define the device
dev = qml.device("lightning.qubit", wires=qubits)
 
# Define the qnode
@qml.qnode(dev)
def circuit(params, wires, s_wires, d_wires, hf_state):
    qml.UCCSD(params, wires, s_wires, d_wires, hf_state)
    return qml.expval(H)
 
# Define the initial values of the circuit parameters
params = np.zeros(len(singles) + len(doubles))
 
# Define the optimizer
optimizer = qml.GradientDescentOptimizer(stepsize=1.0)
 
# Optimize the circuit parameters and compute the energy
for n in range(300):
    params, energy = optimizer.step_and_cost(circuit, params,
    wires=range(qubits), s_wires=s_wires, d_wires=d_wires, hf_state=hf_state)
    if n % 2 == 0:
        print("step = {:},  E = {:.8f} Ha".format(n, energy))
 

doubles are [[0, 1, 4, 5], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 6, 7], [0, 2, 4, 6], [0, 3, 4, 5], [0, 3, 4, 7], [0, 3, 5, 6], [0, 3, 6, 7], [1, 2, 4, 5], [1, 2, 4, 7], [1, 2, 5, 6], [1, 2, 6, 7], [1, 3, 5, 7], [2, 3, 4, 5], [2, 3, 4, 7], [2, 3, 5, 6], [2, 3, 6, 7]]
d wires are [[[0, 1], [4, 5]], [[0, 1], [4, 5, 6, 7]], [[0, 1], [5, 6]], [[0, 1], [6, 7]], [[0, 1, 2], [4, 5, 6]], [[0, 1, 2, 3], [4, 5]], [[0, 1, 2, 3], [4, 5, 6, 7]], [[0, 1, 2, 3], [5, 6]], [[0, 1, 2, 3], [6, 7]], [[1, 2], [4, 5]], [[1, 2], [4, 5, 6, 7]], [[1, 2], [5, 6]], [[1, 2], [6, 7]], [[1, 2, 3], [5, 6, 7]], [[2, 3], [4, 5]], [[2, 3], [4, 5, 6, 7]], [[2, 3], [5, 6]], [[2, 3], [6, 7]]]
step = 0,  E = -1.33256889 Ha
step = 2,  E = -1.64470171 Ha
step = 4,  E = -1.80344401 Ha
step = 6,  E = -1.85817716 Ha
step = 8,  E = -1.87566012 Ha
step = 10,  E = -1.88149918 Ha
step = 12,  E = -1.88357701 Ha
step = 14,  E = -1.88436296 Ha
step = 16,  E = -1.88467749 Ha
step = 18,  E = -1.88480995 Ha
step = 20,  E = -1.88486830 Ha
st

In [5]:
s_wires

[[0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5],
 [1, 2, 3, 4, 5, 6, 7],
 [2, 3, 4],
 [2, 3, 4, 5, 6],
 [3, 4, 5],
 [3, 4, 5, 6, 7]]

In [4]:
d_wires

[[[0, 1], [4, 5]],
 [[0, 1], [4, 5, 6, 7]],
 [[0, 1], [5, 6]],
 [[0, 1], [6, 7]],
 [[0, 1, 2], [4, 5, 6]],
 [[0, 1, 2, 3], [4, 5]],
 [[0, 1, 2, 3], [4, 5, 6, 7]],
 [[0, 1, 2, 3], [5, 6]],
 [[0, 1, 2, 3], [6, 7]],
 [[1, 2], [4, 5]],
 [[1, 2], [4, 5, 6, 7]],
 [[1, 2], [5, 6]],
 [[1, 2], [6, 7]],
 [[1, 2, 3], [5, 6, 7]],
 [[2, 3], [4, 5]],
 [[2, 3], [4, 5, 6, 7]],
 [[2, 3], [5, 6]],
 [[2, 3], [6, 7]]]

In [3]:
## VQE - Exact
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_algorithms import MinimumEigensolver
from qiskit.quantum_info import SparsePauliOp

from qiskit_nature.second_q.operators import SparseLabelOp
from qiskit_nature.second_q.mappers import QubitMapper
from qiskit_nature.second_q.problems import BaseProblem
from qiskit_nature.second_q.problems import EigenstateResult


from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA, QNSPSA, GradientDescent, L_BFGS_B, POWELL
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitMapper

from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import ParityMapper
import numpy as np
from pyscf import gto, scf, mcscf
from qiskit_algorithms import AdaptVQE, AdaptVQEResult




H4 = 'H 0 0 0.0; H 0 0 3.0; H 0 0 6.0; H 0 0 9.0'

n_it = 1
# Specify other molecular properties
charge = 0
spin = 0
basis = 'sto6g'


# Using driver to get fermionic Hamiltonian
# PySCF example
from qiskit.primitives import Estimator
it1 = []
energies_exact= []




optimizer = L_BFGS_B(options={'maxiter': 1e6, 'disp': True, 'maxfun': 1e6})
mapper = JordanWignerMapper()

for i in range(n_it):
        print('The iteration now is ', i)
        driver =  PySCFDriver(atom=H4,
                      unit=DistanceUnit.ANGSTROM,
                      charge=charge,
                      spin=spin,
                      basis=basis)
        qmolecule = driver.run()
        fermionic_op = qmolecule.hamiltonian.second_q_op()
        qubit_op = mapper.map(fermionic_op)
        ansatz = UCCSD(
                qmolecule.num_spatial_orbitals,
                qmolecule.num_particles,
                mapper,
                initial_state = HartreeFock(
                        qmolecule.num_spatial_orbitals,
                        qmolecule.num_particles,
                        mapper,
                        )
                        )
        
        estimator = Estimator()
        vqe = VQE(estimator, ansatz, optimizer, initial_point= [0.0]* ansatz.num_parameters)
        vqe_calcC = vqe.compute_minimum_eigenvalue(qubit_op)
        vqe_result = qmolecule.interpret(vqe_calcC).total_energies[0].real
        energies_exact.append(vqe_result)
        it1.append(i)


print('The angles are',it1)
print("Energies are", energies_exact)
print('VQE Result is',vqe_calcC)


The iteration now is  0


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           26     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -2.09694D+00    |proj g|=  3.10045D-01

At iterate    1    f= -2.63870D+00    |proj g|=  5.59108D-02

At iterate    2    f= -2.64239D+00    |proj g|=  4.28614D-02

At iterate    3    f= -2.64606D+00    |proj g|=  2.82516D-03

At iterate    4    f= -2.64607D+00    |proj g|=  1.83511D-03

At iterate    5    f= -2.64607D+00    |proj g|=  8.95151D-04

At iterate    6    f= -2.64608D+00    |proj g|=  9.68559D-04

At iterate    7    f= -2.64608D+00    |proj g|=  1.74674D-03

At iterate    8    f= -2.64610D+00    |proj g|=  2.73706D-03

At iterate    9    f= -2.64614D+00    |proj g|=  5.28702D-03

At iterate   10    f= -2.64619D+00    |proj g|=  7.49121D-03

At iterate   11    f= -2.64630D+00    |proj g|=  7.59015D-03

At iterate   12    f= -2.64638D+00    |proj g|=  9.48224D-03

At iterate   13    f= -2.6